# FRED Tools Quickstart

This notebook shows the minimum setup for `fredtools` and a few core
calls: series metadata, observations, and release lookup.

## Setup

Set your FRED API key as `FRED_API_KEY` in the environment before running.

In [ ]:
import os

from fredtools import Fred, FredConfig, Series
from fredtools.logging import log_requests

api_key = os.environ.get("FRED_API_KEY")
if not api_key:
    raise RuntimeError(
        "Set FRED_API_KEY in your environment to run this notebook."
    )

log_requests(level="INFO")
client = Fred(FredConfig(api_key=api_key))

## Series Metadata

In [ ]:
gdp = Series("GDP")
gdp

In [ ]:
(gdp.series_id, gdp.title, gdp.frequency, gdp.units)

## Observations

In [ ]:
observations = gdp.observations()
observations[:5]

In [ ]:
try:
    observations.df.head()
except RuntimeError as exc:
    print(exc)

## GDP Revisions Chart

Compare the most recent vintage releases for GDP.


In [ ]:
try:
    import pandas as pd
    import matplotlib.pyplot as plt
except ImportError as exc:
    raise RuntimeError(
        "Install pandas and matplotlib to plot revisions."
    ) from exc

from datetime import date

vintages = gdp.vintage_dates()
if len(vintages) < 2:
    raise RuntimeError("Not enough vintage dates returned for GDP.")

latest_vintage = vintages[-1]
previous_vintages = vintages[:-1]

observation_start = None
max_vintages = None
if max_vintages is not None:
    previous_vintages = previous_vintages[-max_vintages:]

latest_obs = gdp.observations(
    realtime_start=latest_vintage,
    realtime_end=latest_vintage,
    observation_start=observation_start,
)
latest_df = latest_obs.df[["date", "value"]].copy()
latest_df = latest_df.rename(columns={"value": "latest"})

frames = []
for vintage in previous_vintages:
    obs = gdp.observations(
        realtime_start=vintage,
        realtime_end=vintage,
        observation_start=observation_start,
    )
    df = obs.df[["date", "value"]].copy()
    df["vintage"] = vintage
    frames.append(df)

combined = pd.concat(frames, ignore_index=True)
revision_frame = combined.pivot(
    index="date",
    columns="vintage",
    values="value",
)

aligned = revision_frame.join(
    latest_df.set_index("date"),
    how="inner",
)
revision_diff = aligned.drop(columns=["latest"]).rsub(
    aligned["latest"],
    axis=0,
)

revision_diff.tail(200).plot(figsize=(10, 5), legend=False)
plt.title("GDP: Latest vs Previous Revisions")
plt.ylabel("Latest Minus Vintage")
plt.xlabel("Date")
plt.show()


## Release

In [ ]:
release = gdp.release()
(release.release_id, release.name)